### Boilerplate CNN
##### (hopefully with model architecture flexibility)
Most of the following code was taken and adjusted from https://github.com/nikhilroxtomar/Semantic-Segmentation-Architecture/blob/main/PyTorch/unet.py

In [7]:
import torch.nn as nn

In [8]:
# convolutional layer
class cnn_block(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x_in):
        # CNN + relu layer
        x_conv1 = self.conv1(x_in)
        x_relu1 = self.relu(x_conv1)
        # CNN + relu layer
        x_conv2 = self.conv2(x_relu1)
        x_out = self.relu(x_conv2)
        return x_out

# cnn + max pooling block
class conv_pool_block(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()

        self.conv = cnn_block(in_channel, out_channel)
        self.pool = nn.MaxPool2d((2, 2))

    def forward(self, x_in):
        x_out = cnn_block(in_channel, out_channel)
        p_out = self.pool(x_out)

        return x_out, p_out


# upsample
class inverse_conv_block(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()

        self.upsample = nn.ConvTranspose2d(in_channel, out_channel, kernel_size=2, stride=2, padding=0)
        self.conv = cnn_block(out_channel+out_channel, out_channel)

    def forward(self, x_in, skip):
        x_up = self.upsample(x_in)
        x_cat = torch.cat([x_up, skip], axis=1)
        x_out = self.conv(x_cat)
        return x_out

In [9]:
# building the u-net
class build_cnn(nn.Module):
    def __init__(self):
        super().__init__()

        """ convolutions + pooling """
        self.cp1 = conv_pool_block(3, 32)
        self.cp2 = conv_pool_block(32, 64)
        self.cp3 = conv_pool_block(64, 128)
        self.cp4 = conv_pool_block(128, 256)

        """ convolution """
        self.b = cnn_block(256, 512)

        """ inverse convolutions """
        self.ic1 = inverse_conv_block(512, 256)
        self.ic2 = inverse_conv_block(256, 128)
        self.ic3 = inverse_conv_block(128, 64)
        self.ic4 = inverse_conv_block(64, 32)

        """ output """
        self.outputs = nn.Conv2d(32, 1, kernel_size=1, padding=0)

    def forward(self, inputs):
        """ conv + pool """
        x1, p1 = self.cp1(inputs)
        x2, p2 = self.cp2(p1)
        x3, p3 = self.cp3(p2)
        x4, p4 = self.cp4(p3)

        """ conv """
        b = self.b(p4)

        """ upsample """
        u1 = self.ic1(b, x4)
        u2 = self.ic2(u1, x3)
        u3 = self.ic3(u2, x2)
        u4 = self.ic4(u3, x1)

        """ Classifier """
        outputs = self.outputs(u4)

        return outputs

In [10]:
model = build_cnn()